# Imports

In [ ]:
# stdlib
import sqlite3
from datetime import datetime
import subprocess
import os

# 3rd party
import numpy as np
import pandas as pd
import netCDF4 as nc
import tempfile

In [ ]:
# Just for testing
import shutil
from IPython.display import display

# Functions

In [ ]:
def create_table(run_name, prefix='.'):
    """
    Create sqlite db file, create table, and return connection.
    Assume run_name is unique (e.g. includes date.)
    """
    
    create_table_template = '''
    CREATE TABLE {db_name} (

        /* Inputs */
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        absorptance_kelp REAL,
        a_water REAL,
        b REAL,
        ns REAL,
        nz REAL,
        na REAL,
        num_dens REAL,
        kelp_dist CHAR(32),
        fs REAL,
        fr REAL,
        ft REAL,
        max_length REAL,
        length_std REAL,
        zmax REAL,
        rope_spacing REAL,
        date CHAR(64),
        compute_time REAL,
        git_commit CHAR(40),
        num_cores INTEGER,
        num_scatters INTEGER,
        fd_flag INTEGER,
        lis_opts CHAR(256),
        lis_iter INTEGER,
        lis_time INTEGER,
        lis_resid REAL,

        /* Array data (NETCDF) 
        Contains:
          - radiance (ns, ns, nz, na, na) 
          - irradiance (ns, ns, nz)
          - p_kelp (ns, ns, nz)
          - avg_irrad (nz)
          - perc_irrad (nz)
        */
        data_path CHAR(256)
        )
    '''
    
    base_dir = os.path.join(prefix, run_name)
    data_dir = os.path.join(base_dir, 'data')
    db_file = os.path.join(base_dir, run_name+'.db')
    # Will fail if `run_name` is not unique in `prefix`
    os.mkdir(base_dir)
    os.mkdir(data_dir)
    
    conn = sqlite3.connect(db_file)
    conn.execute(create_table_template.format(db_name=run_name))
    
    return conn

In [ ]:
def insert_run(conn, **params):
    insert_template = '''
    INSERT INTO {db_name} VALUES (
        NULL, /* id (autoincrement) */
        {absorptance_kelp},
        {a_water},
        {b},
        {ns},
        {nz},
        {na},
        {num_dens},
        '{kelp_dist}',
        {fs},
        {fr},
        {ft},
        {max_length},
        {length_std},
        {zmax},
        {rope_spacing},
        '{date}',
        {compute_time},
        '{git_commit}',
        {num_cores},
        {num_scatters},
        {fd_flag},
        '{lis_opts}',
        {lis_iter},
        {lis_time},
        {lis_resid},
        '{data_path}'
        )
    '''
    insert_cmd = insert_template.format(**params)
    return conn.execute(insert_cmd)

In [ ]:
def create_nc(data_path, **results):
    # Create file
    rootgrp = nc.Dataset(data_path, 'w', format='NETCDF4')
    
    # Get dimension sizes
    nx = len(results['x'])
    ny = len(results['y'])
    nz = len(results['z'])
    ntheta = len(results['theta'])
    nphi = len(results['phi'])
    
    # Create Dimensions
    x = rootgrp.createDimension('x', nx)
    y = rootgrp.createDimension('y', ny)
    z = rootgrp.createDimension('z', nz)
    theta = rootgrp.createDimension('theta', ntheta)
    phi = rootgrp.createDimension('phi', nphi)
    
    # Create dimension variables
    xvals = rootgrp.createVariable('x','f8', ('x',))
    yvals = rootgrp.createVariable('y','f8', ('y',))
    zvals = rootgrp.createVariable('z','f8', ('z',))
    thetavals = rootgrp.createVariable('theta','f8', ('theta',))
    phivals = rootgrp.createVariable('phi','f8', ('phi',))
    
    # Assign dimension variables
    xvals[:] = results.pop('x')
    yvals[:] = results.pop('y')
    zvals[:] = results.pop('z')
    thetavals[:] = results.pop('theta')
    phivals[:] = results.pop('phi')
    
    # Create results variables
    rad = rootgrp.createVariable('rad', 'f8', ('x', 'y', 'z', 'theta', 'phi'))
    irrad = rootgrp.createVariable('irrad', 'f8', ('x', 'y', 'z'))
    avg_irrad = rootgrp.createVariable('avg_irrad', 'f8', ('z',))
    perc_irrad = rootgrp.createVariable('perc_irrad', 'f8', ('z',))
    p_kelp = rootgrp.createVariable('p_kelp', 'f8', ('x', 'y', 'z'))

    # Assign results variables
    rad[:] = results.pop('rad')
    irrad[:] = results.pop('irrad')
    avg_irrad[:] = results.pop('avg_irrad')
    perc_irrad[:] = results.pop('perc_irrad')
    p_kelp[:] = results.pop('p_kelp')
    
    # Assume all others are scalar metadata
    for var_name, val in results.items():
        var = rootgrp.createVariable(var_name, type(val))
        var[...] = val
    
    # Close file
    rootgrp.close()

In [ ]:
def get_random_unused_filename(dir='.', prefix='', suffix=''):
    with tempfile.NamedTemporaryFile(dir=dir, prefix=prefix, suffix=suffix) as fh:
        filename = fh.name
    return filename

In [ ]:
def get_git_commit_hash():
    return subprocess.check_output(['git','rev-parse','HEAD']).decode().strip()

# Test Functions

In [ ]:
def create_phony_kelp(data_path):
    ns, nz, na = np.random.randint(10, 20, size=3)
    # Make sure na is even
    if na % 2 == 1:
        na -= 1
        
    absorptance_kelp, a_water, b = np.random.rand(3)
    num_dens = np.random.rand()
    fs, fr, ft = np.random.rand(3)
    kelp_dist = 'top-heavy'
    max_length, length_std = np.random.rand(2)
    zmax, rope_spacing = 10 * np.random.rand(2)
    date = datetime.now().ctime()
    compute_time = 100 * np.random.rand()
    git_commit = get_git_commit_hash()
    num_cores = np.random.randint(64)
    num_scatters = np.random.randint(20)
    fd_flag = np.random.randint(2)
    lis_opts = '-i gmres -tol 1e-5'
    lis_iter = np.random.randint(1000)
    lis_time = 100 * np.random.rand()
    lis_resid = 1e-5 * np.random.rand()
    
    return {
        'absorptance_kelp': absorptance_kelp,
        'a_water': a_water,
        'b': b,
        'ns': ns,
        'nz': nz,
        'na': na,
        'num_dens': num_dens,
        'kelp_dist': kelp_dist,
        'fs': fs,
        'fr': fr,
        'ft': ft,
        'max_length': max_length,
        'length_std': length_std,
        'zmax': zmax,
        'rope_spacing': rope_spacing,
        'date': date,
        'compute_time': compute_time,
        'git_commit': git_commit,
        'num_cores': num_cores,
        'num_scatters': num_scatters,
        'fd_flag': fd_flag,
        'lis_opts': lis_opts,
        'lis_iter': lis_iter,
        'lis_time': lis_time,
        'lis_resid': lis_resid,
        'data_path': data_path
    }

In [ ]:
def create_phony_results(ns, nz, na):
    results = {
        # Dimension values
        # These are grid-starts, not grid-centers, but it's phony anyhow.
        'x': np.linspace(0, 1, ns),
        'y': np.linspace(0, 1, ns),
        'z': np.linspace(0, 1, nz),
        'theta': np.linspace(0, 2*np.pi, na),
        'phi': np.linspace(0, np.pi, na),
        
        # Results
        'rad': np.random.rand(ns, ns, nz, na, na),
        'irrad': np.random.rand(ns, ns, nz),
        'p_kelp': np.random.rand(ns, ns, nz),
        'avg_irrad': np.random.rand(nz),
        'perc_irrad': np.random.rand(nz)
    }
    
    return results

In [ ]:
def run_phony(conn, run_name, data_dir):
    data_path = get_random_unused_filename(dir=data_dir, suffix='.nc')
    
    metadata = create_phony_kelp(data_path)
    ns = metadata['ns']
    nz = metadata['nz']
    na = metadata['na']
    results = create_phony_results(ns, nz, na)
    
    create_nc(**metadata, **results)
    insert_run(conn, db_name=run_name, **metadata)

def run_phony_set(run_name, prefix='.', num_entries=10):
    base_dir = os.path.join(prefix, run_name)
    data_dir = os.path.join(base_dir, 'data')
    conn = create_table(run_name, prefix)
    for i in range(num_entries):
        run_phony(conn, run_name, data_dir)
        
    return conn

In [ ]:
def table_to_df(conn, table_name):
    select_cmd = 'SELECT * FROM {table_name}'.format(table_name=table_name)
    cursor = conn.execute(select_cmd)
    columns = [col[0] for col in cursor.description]
    data = [x for x in cursor]
    
    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
def query_results(conn, table_name, **kwargs):
    # Sanitize string values
    for key, val in kwargs.items():
        if isinstance(val, str):
            kwargs[key] = '"{}"'.format(val)
    
    # Form SQL WHERE clause
    where_clause = ' AND '.join([
        '{}={}'.format(key, value)
        for key, value in kwargs.items()
    ])
    
    # Form entire query
    query = '''SELECT data_path FROM {table_name}
    WHERE {where_clause}
    '''.format(
        table_name=table_name,
        where_clause=where_clause
    )
    
    # Execute query
    cursor = conn.execute(query)
    results_list = [x for x in cursor]
    
    # Extract matching datasets
    datasets = [nc.Dataset(results[-1]) for results in results_list]
    return datasets

# Test

In [ ]:
run_name = 'test1'
prefix='.'
base_dir = os.path.join(prefix, run_name)

# Remove dir if it exists
shutil.rmtree(base_dir, ignore_errors=True)

# Run set
conn = run_phony_set(run_name, prefix=prefix, num_entries=10)
display(table_to_df(conn, run_name))

# Look at results

In [ ]:
ds, *_ = query_results(conn, 'test1', kelp_dist='top-heavy', id=1)

print(ds['z'])
print(ds['z'][:])

print(ds['perc_irrad'])
print(ds['perc_irrad'][:])

print(ds['compute_time'])
print(ds['compute_time'][:])

print(ds['git_commit'])
print(ds['git_commit'][:])